In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
import  analysis.lib.purification.purify_pq as ppq

%matplotlib inline

thetas =['pi/4','pi/5','pi/6','pi/8']

outcome_bins = np.arange(50,540,50)

In [ ]:
def calculate_ebit_rate(outcome_bins,apply_ROC = True):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
    
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    total_counts_p = psi_p_XX[0]+psi_p_YY[0]+psi_p_ZZ[0]
    total_counts_m = psi_m_XX[0]+psi_m_YY[0]+psi_m_ZZ[0]
    
    total_time = Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    rate_p = np.array(total_counts_p)/total_time
    rate_m = np.array(total_counts_m)/total_time
    
    ebit_rate_p = np.sum(rate_p*ebits_p)
    ebit_rate_p_u = np.sqrt(np.sum(rate_p**2*ebits_p_u**2))
    ebit_rate_m = np.sum(rate_m*ebits_m)
    ebit_rate_m_u = np.sqrt(np.sum(rate_m**2*ebits_m_u**2))
    
    avg_fidelity_p = np.sum(psi_p_F*total_counts_p)/np.sum(total_counts_p)
    avg_fidelity_p_u = np.sqrt(np.sum(psi_p_F_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_fidelity_m = np.sum(psi_m_F*total_counts_p)/np.sum(total_counts_m)
    avg_fidelity_m_u = np.sqrt(np.sum(psi_m_F_u**2*total_counts_p**2))/np.sum(total_counts_m)
    
    avg_ebits_p = np.sum(ebits_p*total_counts_p)/np.sum(total_counts_p)
    avg_ebits_p_u = np.sqrt(np.sum(ebits_p_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_ebits_m = np.sum(ebits_m*total_counts_m)/np.sum(total_counts_p)
    avg_ebits_m_u = np.sqrt(np.sum(ebits_m_u**2*total_counts_m**2))/np.sum(total_counts_p)
    
#     fig  = plt.figure()
#     ax = plt.subplot()
#     ax.set_xlabel('Superposn ms=0 frac.')
#     ax.set_ylabel('Rate')
#     plt.plot(outcome_bins,rate_p)
#     plt.plot(outcome_bins,rate_m)
#     plt.show()
#     plt.close('all')

    rate_p = np.sum(rate_p)
    rate_m = np.sum(rate_m)
    
    return [rate_p,rate_m,ebit_rate_p,ebit_rate_p_u,ebit_rate_m,ebit_rate_m_u,avg_fidelity_p,avg_fidelity_p_u,avg_fidelity_m,avg_fidelity_m_u,avg_ebits_p,avg_ebits_p_u,avg_ebits_m,avg_ebits_m_u]

In [ ]:
vals = []

for theta in thetas:
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')

#     ### load the data
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    
    vals.append(calculate_ebit_rate(outcome_bins = outcome_bins))
   
vals = np.transpose(vals)

## Calculations for other bounds

In [ ]:
TPQI_vis = 0.5765
TPQI_vis_u = 0.04

estimatedFirstClickProb = 2 * 0.000715 # Multiply by two since at 50% superpos.
tailLT3 = 10.
tailLT4 = 5.
firstClickProbLT3 = (tailLT4/(tailLT4+tailLT3))*estimatedFirstClickProb
firstClickProbLT4 = (tailLT3/(tailLT4+tailLT3))*estimatedFirstClickProb

firstClickProbLT3_u = 0.2*firstClickProbLT3
firstClickProbLT4_u = 0.2*firstClickProbLT4

pBK = 0.5*firstClickProbLT3*firstClickProbLT4
estimatedRateBK = pBK / (7e-6)

estimatedRateBK_u = ( 0.5 / (7e-6))*np.sqrt(firstClickProbLT3**2*firstClickProbLT4_u**2+firstClickProbLT4**2*firstClickProbLT3_u**2)
estimatedEbitsBK= np.log2(1+TPQI_vis)
estimatedEbitsBK_u = TPQI_vis_u/((1+TPQI_vis)*np.log(2))
# print pBK
# print estimatedRateBK
# print estimatedEbitsBK

estimatedEbitRateBK = estimatedRateBK*estimatedEbitsBK
estimatedEbitRateBK_u = np.sqrt(estimatedEbitsBK_u**2 * estimatedRateBK**2 + estimatedEbitsBK**2 * estimatedRateBK_u**2)

In [ ]:
pur_model_populations = np.array([0.0244717, 0.0380602, 0.0544967, 0.0736799, 0.0954915, 0.119797, \
0.146447, 0.175276, 0.206107, 0.238751, 0.273005, 0.308658, 0.345492, \
0.383277, 0.421783, 0.46077, 0.5, 0.53923, 0.578217, 0.616723, \
0.654508, 0.691342, 0.726995])

pur_model_ebit_rate = np.array([0.00171524, 0.0041989, 0.00836286, 0.0144633, 0.0225308, 0.0323439, \
0.0434359, 0.0551244, 0.0665555, 0.0768076, 0.084981, 0.0902961, \
0.0921844, 0.0903651, 0.0849169, 0.0761971, 0.0648926, 0.0519372, \
0.0384232, 0.0254953, 0.0142381, 0.00556886, 0.000880181])

In [ ]:
thetas = [np.pi/4,np.pi/5,np.pi/6,np.pi/8]

rate_p = vals[0]
rate_m = vals[1]
ebit_rate_p = vals[2]
ebit_rate_p_u = vals[3]
ebit_rate_m = vals[4]
ebit_rate_m_u = vals[5]
avg_fidelity_p = vals[6]
avg_fidelity_p_u = vals[7]
avg_fidelity_m = vals[8]
avg_fidelity_m_u = vals[9]
avg_ebits_p = vals[10]
avg_ebits_p_u = vals[11]
avg_ebits_m = vals[12]
avg_ebits_m_u = vals[13]

super_posn_frac = np.sin(thetas)**2

total_ebit_rate = ebit_rate_p + ebit_rate_m
total_ebit_rate_u = np.sqrt(ebit_rate_p_u**2 + ebit_rate_m_u**2)

real_theta_vals = np.flipud((0.5-np.array([0.153355,0.2525,0.2827,0.323754,0.362197,0.3865]))
)*np.pi
real_pop_vals = np.sin(real_theta_vals)**2
#needs reversing:
real_pop_vals = real_pop_vals[::-1]


save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*89
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

lw = 1
fontsize = 10
markersize = 3

ax.set_xlabel(r'Population in ms=0')
ax.set_ylabel('Ebit rate (Hz)')
#ax.set_yscale("log", nonposy='clip')

plt.plot(pur_model_populations,pur_model_ebit_rate,linestyle='--',color='#50A53A',lw=lw)
plt.errorbar(real_pop_vals[1:-1],total_ebit_rate,total_ebit_rate_u,marker='o',ms=markersize,mew =1,capsize=3, 
                     elinewidth=lw,ls='none',label = 'Pur',color='#50A53A')
# plt.plot([0, 0.6], [estimatedEbitRateBK, estimatedEbitRateBK], label = 'BK')
plt.plot([0, 0.6],np.ones(2)* estimatedEbitRateBK, 'k', color='k',label = 'BK',lw=lw)
plt.fill_between([0, 0.6],np.ones(2)*(estimatedEbitRateBK-estimatedEbitRateBK_u),np.ones(2)*( estimatedEbitRateBK+estimatedEbitRateBK_u),
    alpha=0.3, edgecolor='k', facecolor='k')

plt.plot([0, 0.6],np.ones(2)* estimatedRateBK, '--', color='k',label = 'BK perfect',lw=lw)
# plt.fill_between([0, 0.6],np.ones(2)*(estimatedRateBK-estimatedRateBK_u),np.ones(2)*( estimatedRateBK+estimatedRateBK_u),
#     alpha=0.5, edgecolor='#3F7F4C', facecolor='#7EFF99')

plt.xlim([0.1,0.55])
plt.ylim([0.0,1.2*plt.ylim()[1]])
# plt.legend( numpoints = 1,loc=3)

###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)



plt.savefig(os.path.join(save_folder,'ebit_rate.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'ebit_rate.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')

# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Raw rate (Hz)')
# plt.plot(super_posn_frac,rate_m,label = '-')
# plt.plot(super_posn_frac,rate_p,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')

# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Avg fidelity')
# plt.errorbar(super_posn_frac,avg_fidelity_m,avg_fidelity_m_u,label = '-')
# plt.errorbar(super_posn_frac,avg_fidelity_p,avg_fidelity_p_u,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')


# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Avg ebits')
# plt.errorbar(super_posn_frac,avg_ebits_m,avg_ebits_m_u,label = '-')
# plt.errorbar(super_posn_frac,avg_ebits_p,avg_ebits_p_u,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')

In [ ]:
print total_ebit_rate
print real_pop_vals[1:-1]
print super_posn_frac

In [ ]:
plt.errorbar?